# Particle Filter with Julia

## Introduction

In time series models that have latent variables (i.e. have unobservable variables), particle filters (PF) are used to estimate the path of the latent variables up to time $t$ using only information available at time $t$. This operation is called filtering, and it is what gives the PF its name (or at least, part of it). But PF can also be used to estimate the path of the latent variables using all the data, an operation that is called smoothing. Last, but not least, PF are useful to integrate the unobservable variables out of the joint density of observable and unobservable variables to get the marginal density of the observable variables---the likelihood, which can be used to estimate the unknown parameter of the model by using standard Bayesian computational tools.

This Julia notebook describes the PF algorithms and its implementation in Julia. The objective is not to write efficient code, but to provide a pedagogical overview of the PF based on actual code. 

## Requirement

In these notes I am assuming the use of julia-0.4.5. With earlier (or later version) the code below is not guaranteed to excecute without wome twicking. 

The timings presented are relative to this machine.

In [1]:
versioninfo()

Julia Version 0.5.1
Commit 6445c82 (2017-03-05 13:25 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin13.4.0)
  CPU: Intel(R) Core(TM) i7-6820HQ CPU @ 2.70GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.7.1 (ORCJIT, broadwell)


This notebook requires the following packages to be installed and loadable on your installation of Julia. 

In [2]:
pkgs = ["Distributions", "StatsBase", "StatsFuns", "Plots"]
ver = [Pkg.installed(u) for u in pkgs];
[pkgs ver]

4×2 Array{Any,2}:
 "Distributions"  v"0.12.2"
 "StatsBase"      v"0.13.1"
 "StatsFuns"      v"0.4.0" 
 "Plots"          v"0.10.3"

In [ ]:
using Distributions
using StatsBase
using StatsFuns
using Plots

## Generic model


Particle methods assume $\{X_{t}\}_{t\geq1}$ and the observations $\{Y_{t}\}_{t\geq1}$ satisfy the following setup:


1. $X_{1},X_{2},\ldots$ is a first order Markov process such that 
$$
X_{t}|\left(X_{t-1}=x_{t-1}\right)\sim f_{\theta}(\cdot|x_{t-1})
$$
with an initial distribution
$$
X_{1}\sim\mu_{\theta}(x_{1}).
$$
 
2. Given $\{X_{t}\}_{t\geq1}$, the observations $\{Y_{t}\}_{t\geq1}$
are statistically independent, and their marginal conditional densities are given
$$
Y_{t}|\left(X_{t}=x_{t}\right)\sim g_{\theta}(y_{t}|x_{t}),
$$

A model with these characteristics is the Stochastic Volatility model. 

## Stochastic Volatility

The stochastic volatility model:

\begin{align*}
X_{t} & =\alpha X_{t-1}+\sigma V_{t}\\
Y_{t} & =\beta\exp\left(X_{t}/2\right)W_{t}
\end{align*}
where
$$
V_{t}\sim N(0,1),\quad W_{t}\sim N(0,1).
$$

In the notation of the generic model, we have:

$$
\theta=(\alpha,\sigma,\beta),\,\,\mu_{\theta}(x)=N\left(x;0,\frac{\sigma^{2}}{1-\alpha^{2}}\right),
$$

$$
f_{\theta}(x_{t}|x_{t-1})=N(x_{t};\alpha x_{t-1},\sigma^{2})
$$
and
$$
g_{\theta}(y_{t}|x_{t})=N\left(y;0,\beta^{2}\exp(x)\right).
$$

## Simulating the SV model

We start by simulating a series $Y$ from the SV model using as parameters 
$$
\alpha_0 = 0.91,\,\ \sigma_0 = 1\,\, \beta_0 = 0.5.
$$

We let $\theta = [\alpha, \sigma, \beta]$, and $\theta_0 = [0.91, 1.0, 0.5]$.



In [ ]:

function simSV(θ, T=100)
    α, σ, β = θ
    println("Simulating SVM")
    println("α = ", α, ", σ = ", σ, ", β = ", β)    
    X = Array(Float64, T + 1)
    Y = Array(Float64, T)
    X[1] = 0.0
    for t = 1:T
        X[t+1] = α*X[t] + σ*randn()
        Y[t]   = β*exp(X[t]/2)*randn()
        
    end
    return Y, X
end
    
θ = [.91, 1.0, 0.5];
srand(1)
Y, X_true = simSV(θ, 500);

We plot the observations (Y) and the volatility (X). 

In [ ]:
Plots.plot(X_true[2:end], color = :blue, lab = "Volatility")
Plots.plot!(Y, color = :black, lab = "Observations")

## Particle filter algorithm
The generic algorithm can be described as follows.

---

At time $t=1$

- Sample $X_{1}^{i}\sim q_{1}(x_{1})$
- Compute the weights $$w_{1}(X_{1}^{i})=\frac{\mu(X_{1}^{i})g(y_{1}|X_{1}^{i})}{q(X_{1}^{i})}$$
and $$W_{1}^{i} = \frac{w_{1}(X_{1}^{i})}{\sum_{i=1}^N w_{1}(X_{1}^{i})}$$

At time $t\geq2$


- Sample $X_{t}^{i}\sim q_{t}(x_{t}|X_{1:t-1}^{i})$ 
- Compute the weights
$$w_{t}(X_{1:t}^{i})  =w_{t-1}(X_{1:t-1}^{i}){\alpha_{t}(X_{1:t}^i)}$$
where 
$$
\alpha_{t}(X_{1:t}^{i})=\frac{g_{\theta}(y_{t}|X_{t}^{i})f_{\theta}(X_{t}^{i}|X_{t-1}^{i})}{q_{t}(X_{t}^{i}|X_{1:t-1}^{i})}
$$

---

At each $t$, the sum of the weights (over $i$) provides an estimate of $p_{\theta}(y_t|y_{1:t-1})$:
$$
\hat{p}_{\theta}(y_t|y_{1:t-1}) = \frac{1}{N}\sum_{i=1}^N w_{t}(X_{1:t}^{i}).
$$

Since
$$
p_{\theta}(y_{1:T}) = p_{\theta}(y_1)\prod_{t=2}^T p_{\theta}(y_t|y_{1:t-1})
$$
we have
$$
\hat{p}_{\theta}(y_{1:T}) = \prod_{t=1}^T \frac{1}{N}\sum_{i=1}^N w_{t}(X_{1:t}^{i}).
$$
Importantly, it can be shown that this estimator of the likelihood function is unbiased, that is, 
$$
E\left[\hat{p}_{\theta}(y_{1:T})\right] = p_{\theta}(y_{1:T}).
$$

The proposal distribution, generically denoted $q_{t}(X_{t}^{i}|X_{1:t-1}^{i})$ in the algorithm above, needs to be choosen to make the algorithm feasible. A common choice is to set
$$
q_{t}(X_{t}^{i}|X_{1:t-1}^{i}) = f_{\theta}(X_{t}|x_{t-1}).
$$
With this choice
$$
\alpha_{t}(X_{1:t}) = g_{\theta}(y_t|x_t).
$$

The function `SIS` below implement the PF just described.

In [ ]:
function SIS(θ, Y, N=1000)
    T = size(Y,1)
    X = Array(Float64, N, T) ## This will store the particles
    W = Array(Float64, N, T) ## This will store the weights
    llik = 0.0
    α, σ, β = θ
    ## Initialize -- Draws particles
    rand!(Normal(0, sqrt(σ^2/(1-α^2))), view(X, :, 1)) ## X_1 ~ μ
    sw = 0.0
    @inbounds for i = 1:N
        W[i, 1] = pdf(Normal(0, β*exp(X[i]/2)), Y[1]) ## W_1 = g(y|X_i)
        sw += W[i,1]
    end
    llik += log(sw)-log(N)
    for j = 2:T
        sw = 0.0        
        @inbounds for i = 1:N
            ## Generate particles
            X[i,j] = rand(Normal(α*X[i, j-1], σ))
            ## Update weights
            W[i,j] = W[i,j-1]*pdf(Normal(0, β*exp(X[i,j]/2)), Y[j])
            ## sw will contain sum of the weights
            sw += W[i,j] 
        end
        llik += log(sw)-log(N)
    end    
    (scale!(W, vec(1./sum(W, 1))), X, llik)
end
        
    
    

The function takes three arguments: $\theta$, $Y$, and $N$. The first argument is the parameter vector. The second is the vector of data. The last is the number of particles to be used.

The algorithm is relatively fast---even if it is written with readibility rather than efficiency in mind. With $N=1000$, `SIS` takes less than 0.1 second to finish.

In [ ]:
W, X, llik = SIS([0.9, 1., .5], Y, 1000);
@time W, X, llik = SIS([0.9, 1., .5], Y, 1000);

To verify the quality of the estimate of the volatility, we plot the "true" value of the volatility (that we have here because we are generating the data) and the estimates produced by the PF. As estimates, we use the weighted mean of the particles at each time $t$. 

In [ ]:
X_est = Array(Float64, size(X,2))
V_est = Array(Float64, size(X,2))
for t = 1:size(X,2)
    X_est[t] = mean(X[:,t], weights(W[:,t]))
    V_est[t] = var(X[:,t], weights(W[:,t]))
end



Notice that both  `X_est` and `W_est` are $T\times N$ matrices. The column $t$ of $W_est$ contains the particles at time $t$ while `W_est` contains the time $t$ weights. This means that `X_est[t] = mean(X[:,t], weights(W[:,t]))` estimates the mean of the volatility at time $t$ given $y_1,\ldots,y_t$, or
$$
\bar{x}_{t|1:t} = \int x_t \hat{p}_{\theta}(x_t|y_{1:t}) d x_t = \sum_{i=1}^N x_{t}^i W_t^i.
$$

Similarly, `V_est[t] = var(X[:,t], weights(W[:,t]))` calculates
$$
\int (x_t - \bar{x}_{t|1:t})^2 \hat{p}_{\theta}(x_t|y_{1:t}) d x_t = \sum_{i=1}^N (x_{t}^i-\bar{x}_{t|1:t})^2 W_t^i.
$$

Below, we plot both the true and the filtered volatility and the credibility bands for $t=1,\ldots,100$.

In [ ]:
Plots.plot(X_true[1:100], color = :red, lab = "Volatility")
Plots.plot!(X_est[1:100], color = :blue, lab = "Estimated volatility")
Plots.plot!(X_est[1:100] + sqrt(V_est[1:100]), line=:dot, color = :blue, lab = "+/- 1 SD")
Plots.plot!(X_est[1:100] - sqrt(V_est[1:100]), line=:dot, color = :blue, lab = "")


The quality of the filter is good initially, but it degrades quickly. For $t=50$ onward, the filtered states fail to track the true volatility, and the bands are narrow. Eventually, the filter collapses for larger values of $t$. The reason for the bad performance is the depletion of the particles. The graph below plots the histograms of the particles at $t=1$, $t=5$, $t=20$ and $t=50$. The histograms show that already at $t=20$ only a few particles receive positive weight while at $t=50$ there is only one active particle.

In [ ]:
Plots.plot(W[:,1];  t = :histogram, title = "t = 1", lab = "", layout = 4)
Plots.plot!(W[:,5],  t = :histogram, title = "t = 5", lab = "", subplot = 2)
Plots.plot!(W[:,20], t = :histogram, title = "t = 20", lab = "", subplot = 3)
Plots.plot!(W[:,50], t = :histogram, title = "t = 50", lab = "", subplot = 4)


At $t$ grows, eventually only one particle survives, and the log-likelihood becomes $-\infty$. 


A way to obviate to the depletion problem is to add a resampling step in the filter. A description of the algorithm with the resempling step is below.

---

At time $t=1$

- Sample $X_{1}^{i}\sim q_{1}(x_{1})$
- Compute the weights $$w_{1}(X_{1}^{i})=\frac{\mu(X_{1}^{i})g(y_{1}|X_{1}^{i})}{q(X_{1}^{i}|y_{1})}$$
and $$W_{1}^i = \frac{w_{1}(X_1^i)}{\sum_{i=1}^N w_{1}(X_1^i)}$$
- Resample $X_{1}^{i}$ with probabilities $W_{1}^{i}$. Denote the resampled particle $\bar{X}_{1}^i$.

At time $t\geq2$


- Sample $X_{t}^{i}\sim q_{t}(x_{t}|\bar{X}_{1:t-1}^{i})$ and set $X_{1:t}^i = (\bar{X}_{1:t-1}^i, X_t^i)$
- Compute the weights $w_{t}(X_{1:t}^i)  = \alpha_t(X_{1:t}^i)$ and $W_t^i = \frac{w_{t}(X_{1:t}^i)}{\sum_{i=1}^N w_{t}(X_{1:t}^i)}$
- Resample $X_{1:t}^i$ with probabilities $W_t^i$ to obtain $\bar{X}^i_{1:t}$.
---


There many ways to carry-out the resampling step. We use here the systematic resampling algorithm. 

The function `SIR` implements the algorithm just described. The function returns two output: the particles, `X`, and the estimated log-likelihood. 

In [ ]:
""" Performs the systemic resampling algorithm used by particle filters.
Parameters
----------
indexes   : N x 1 Array{Int}
            array of indexes into the weights defining the resample.
            i.e. the index of the first resample is indexes[1], etc.
weights   : N x 1 Array{Float64, 1} weights (sum 1)
positions : N x 1 Array{Float64, 1} will contain the positions
cumsums   : N x 1 Array{Float64, 1} will contain the cumulative sum
-------
"""
function systematic_resample!(indexes, w, positions, cumsums)
    N = length(w)
    U1 = rand(1)[1]/N
    @simd for j in 1:N
        @inbounds positions[j] = U1 + (j-1)/N
    end
    cumsum!(cumsums, w)
    i, j = 1, 1
    @inbounds while i <= N
        if positions[i] < cumsums[j]
            indexes[i] = j
            i += 1
        else
            j += 1
        end
    end
    return indexes
end

function SIR(θ, Y, N=1000)
    T = size(Y,1)
    X = Array{Float64}(N, T) ## This will store the particles
    w = Array{Float64}(N)
    Xc = Array{Float64}(N)
    cumsums = Array{Float64}(N)
    indexes = Array{Int64}(N)
    positions = Array{Float64}(N)
    α, σ, β = θ
    ## Initialize -- Draws particles
    randn!(view(X, :, 1))
    X[:,1] .*= sqrt(σ^2/(1-α^2))
    @inbounds for i = 1:N
        w[i] = normpdf(0, β*exp(X[i]/2), Y[1]) ## W_1 = g(y|X_i)
    end
    ## Calculate p(y_1)
    llik = log(mean(w))
    w ./= sum(w)
    ## Resample step
    systematic_resample!(indexes, w, positions, cumsums)
    X[:,1] = X[indexes,1]
    @inbounds for j = 2:T
        for i in 1:N
            X[i,j] = α*X[i, j-1] + randn()*σ
            w[i] = normpdf(0, β*exp(X[i, j]/2), Y[j])
        end
        llik += log(mean(w))
        w ./= sum(w)
        systematic_resample!(indexes, w, positions, cumsums)
        X[:,1:j] = X[indexes, 1:j]
    end
    (X, llik)
end


In [ ]:
X, llik = SIR(θ,Y, 1000);

To verify the ability of the filter to estimate the state variables $X$, we plot the true states and the average of the particles. We also plot $\pm 2$ standard errors calculated as the squared root of the variance of the particles. As it can be seen below, the approximation is much better than the filter without the resampling step. 

In [ ]:
X_est = mean(X, 1)'
V_est = var(X, 1)'
Plots.plot(X_true[1:500], color = :red, lab = "Volatility")
Plots.plot!(X_est[1:500], color = :blue, lab = "Estimated volatility")
Plots.plot!(X_est[1:500] + 2*sqrt(V_est[1:500]), line=:dot, color = :blue, lab = "+/- 1 SD")
Plots.plot!(X_est[1:500] - 2*sqrt(V_est[1:500]), line=:dot, color = :blue, lab = "")


## Particle MCMC

Bluntly put, the pMCMC consist in using the marginal likelihood obtained from the particle filter as the likelihood in a Metropolis-Hastings algorithm to obtain draws from the posterior of $\theta$
$$
p(\theta|y_{1:T}) \propto p_{\theta}(Y_{1:T})p(\theta)
$$
A key result is that the particle filter estimator of the likelihood $\hat{p}_{\theta}(y_{1:T})$ is an unbiased estimator of the actual likelihood, that is, it holds
$$
E[\hat{p}_{\theta}(y_{1:T})] = p_{\theta}(y_{1:T}).
$$

The function `SIR` returns (the second argument) the log-likelihood evaluated at a given value of $\theta$. However, the function is too slow for the purpouse of MC simulations. 

In [ ]:
@time X, llik = SIR(θ, Y, 1000);

The function allocates too much memory because to keep track of all the particles matrices have to used and these in turns allocate lots of memory in the resampling step. Since for the MCMC we do not need the particles as output we can be clever and using arrays. This function is called `llikSIR` and it is given below.

In [ ]:
function llikSIR(θ, Y, N = 1000)
    T = size(Y,1)
    X  = Array{Float64}(N) ## This will store the particles at time t
    X′ = Array{Float64}(N) ## This will store the particles at time t′
    w = Array{Float64}(N)
    cumsums = Array{Float64}(N)
    indexes = Array{Int64}(N)
    positions = Array{Float64}(N)
    α, σ, β = θ
    ## Initialize -- Draws particles
    rand!(Normal(0, sqrt(σ^2/(1-α^2))), X)
    @inbounds for i = 1:N
        w[i] = normpdf(0, β*exp(X[i]/2), Y[1]) ## W_1 = g(y|X_i)
    end
    ## Calculate likelihood
    sumw = sum(w)
    llik = log(sumw)
    w ./= sumw
    ## Resample step
    systematic_resample!(indexes, w, positions, cumsums)
    @inbounds for j in 1:N
        X′[j] = X[indexes[j]]
    end
    @inbounds for j = 2:T
        for i in 1:N
            X[i] = α*X′[i] + randn()*σ
            w[i] = normpdf(0, β*exp(X[i]/2), Y[j])
        end
        sumw = sum(w)
        llik += log(sumw)
        w ./= sumw
        if sumw > 0
            systematic_resample!(indexes, w, positions, cumsums)
        else
            break
        end
        for j in 1:N
            X′[j] = X[indexes[j]]
        end
    end
    llik - T*log(N)
end



Calculate of $\hat{p}_{\theta}(y_{1:T})$ is relatively fast for this model. 

As it can be seen belowm this code is 15 times faster than the previous version. At this speed, we can run 1000 iterations of the MCMC in about 70 seconds. 

In [ ]:
llikSIR([.9, 1., .5], Y, 1000);
@time llikSIR([.9, 1., .5], Y, 1000);

In estimating the parameters we elicit two uniform prior for $\alpha$ and $\beta$ and a Gamma prior for $\sigma$. These priors are relatively uninformative. The MCMC is a random walk metropolis algorithm. As usual we have to set the scale of the proposal distribution and the parameter $\tau$. The code below implement the MCMC sampler. 

In [ ]:
πσ = Gamma(2,1)     ## prior on σ
πα = Uniform(-1,1)  ## pr ior on α
πβ = Uniform(0,1)   ## prior in β

## n_particle
n_particle = 1000

function metroprw(n_draw, theta0, Sigma, tau, Y, n_particle = 1000)
    k = length(theta0)
    NormDist = MvNormal(zeros(k), Sigma)
    ## Initialization
    theta      = zeros(k, n_draw+1)
    gamma_s    = logposterior(theta0, Y, n_particle)
    theta[:,1] = theta0
    for s=1:n_draw
        theta_star =  theta[:,s] + tau*rand(NormDist, 1)
        rem(s, 100) == 0 && println("Iteration $s Status: $theta_star Likelihood: $gamma_s")
        gamma_star = logposterior(theta_star, Y, n_particle)
        r = min(exp(gamma_star-gamma_s),1)
        u = rand()
        if(u<=r)
            theta[:, s+1] = theta_star
		          gamma_s = gamma_star
        else
            theta[:, s+1] = theta[:,s]
        end
     end
     return theta
end

function logposterior(theta, Y, n_particle)
    α,σ,β = theta
    logprior = logpdf(πσ, σ) + logpdf(πβ, β) + logpdf(πα, α)
    isfinite(logprior) ? llikSIR(theta, Y, n_particle) + logprior : -Inf     
end

    


In [ ]:
logposterior(θ, Y, 10000)

In [ ]:
out = metroprw(1000, θ, 0.1*eye(3), .3, Y)

In [ ]:
Plots.histogram(out', layout = 3)